# 많은 함수를 동시에 실행하려면 코루틴을 고려하자

스레드를 사용하여 여러 기능을 동시에 실행하는 것 처럼 보여줄 수 있다. 하지만 스레드를 사용하는 데는 세 가지 문제점 존재

1. 스레드를 사용하는 코드가 절차적인 싱글 스레드 코드보다 이해가 어려워 확장성이나 유지보수가 어렵다.
2. 스레드에 메모리가 많이 필요하다.
3. 스레드를 시작하는 데는 비용이 많이 든다. 

파이썬에서는 `coroutine`으로 이런 문제를 모두 해결한다. 코루틴은 제네레이터를 확장하는 방법으로 구현한다. 제네레이터 코루틴을 시작하는 데 드는 비용은 함수 호출이다. 따라서 한 번 활성화되면 소진 시까지 1KB 미만의 메모리만 소비한다.

코루틴은 제네레이터를 소비하느 코드에서 send 함수를 사용하여 역으로 제너레이터 함수의 각 yield 표현식에 값을 보낼 수 있게 하는 방법으로 동작한다.

In [16]:
def my_coroutine():
    while True:
        received = yield
        print('Received:', received)
        
it = my_coroutine()
next(it)
it.send('First')
it.send('Second')

Received: First
Received: Second


yield와 send의 조합은 제너레이터가 외부 입력에 반응하여 다음에 다른 값을 얻게 하는 표준 방법

In [17]:
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)

In [18]:
it = minimize()
next(it)
print(it.send(10))
print(it.send(4))
print(it.send(22))
print(it.send(-1))

10
4
4
-1


코루틴은 스레드와 마찬가지로 주변 환경에서 받은 입력을 소비하여 결과를 만들어낼 수 있는 독립적인 함수다. 둘의 차이는 코루틴이 제너레이터 함수의 각 yield 표현식에서 멈췄다가 외부에서 send를 호출할 때마다 다시 시작한다는 점이다.

덕분에 제너레이터를 소비하는 코드에서 코루틴의 각 yield 표현식 이후에 원하는 처리를 할 수 있다. 제너레이터를 소비하는 코드는 제너레이터의 출력값으로 다른 함수를 호출하고 자료 구조를 수정할 수 있다.

In [19]:
from docutils import namedtuple

Query = namedtuple('Query', ('y', 'x'))

In [24]:
def count_neighbors(y, x):
    n_ = yield Query(y + 1, x + 0)
    ne = yield Query(y + 1, x + 1)
    e_ = yield Query(y + 0, x + 1)
    se = yield Query(y - 1, x + 1)    
    s_ = yield Query(y - 1, x + 0)
    sw = yield Query(y - 1, x - 1)
    w_ = yield Query(y + 0, x - 1)    
    nw = yield Query(y + 1, x - 1)
    
    neighbor_states = [n_, ne, e_, se, s_, sw, w_, nw]
    count = 0
    for state in neighbor_states:
        if state == ALIVE:
            count += 1
    return count

In [25]:
it = count_neighbors(10, 5)
q1 = next(it)
print('Frist yield:  ', q1)
q2 = it.send(ALIVE)
print('Second yield: ', q2)
q3 = it.send(ALIVE)
print('Thrid yield:  ', q3)

try:
    count = it.send(EMPTY)
except StopIteration as e:
    print('Count: ', e.value)

Frist yield:   Query(y=11, x=5)


NameError: name 'ALIVE' is not defined